In [1]:
import os
import numpy as np
from quantum_model import Quantum_Strategy

In [54]:
n = 100
test_size = 1000
qs = Quantum_Strategy(n)
data = np.load(f"./data/data_{n}.npz")
x_train = data['X']; y_train = data['Y']
x = np.random.randint(0, 2, (test_size, x_train.shape[1]))

In [55]:
def encoder(x):
    length = x.shape[0]
    x = x.reshape(length, -1, 2)
    x = np.sum(x * (2 ** np.arange(x.shape[-1])[::-1]).reshape(1, 1, -1), axis=-1)
    x = x.reshape(length, 2, -1)
    return x

In [58]:
def decoder_base(x):
    x1 = x[0]; x2 = x[1]
    y1 = []
    y2 = []
    for i in range(len(x1)):
        if x1[i] == 0 or x2[i] == 0:
            y1.append(np.array([0, 0]))
            y2.append(np.array([0, 0]))
            continue
        x1[i] = x1[i] - 1
        x2[i] = x2[i] - 1
        y_1 = np.zeros(3, dtype=np.int32)
        y_2 = np.zeros(3, dtype=np.int32)
        y_1[2] = 1
        y_2[x1[i]] = y_1[x2[i]]
        if x1[i] == 2:
            y_2[1] = ((y_2[0] + y_2[2])) % 2
        y1.append(y_1[:2])
        y2.append(y_2[:2])
    y1 = np.array(y1).flatten()
    y2 = np.array(y2).flatten()
    return np.concatenate([y1, y2])

def decoder(x):
    y = []
    for i in range(x.shape[0]):
        y.append(decoder_base(x[i]))
    return np.array(y)

In [59]:
np.mean(qs.check_input_output(x_train, decoder(encoder(x_train)), flatten=False))

1.0